## Visualizing Streaming Data with Plotly

### The Basics

In [ ]:
import zmq
from datetime import datetime
import plotly.graph_objects as go 

In [ ]:
symbol = 'SYMBOL'

In [ ]:
fig = go.FigureWidget()
fig.add_scatter()

In [ ]:
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect('tcp://0.0.0.0:5555')
socket.setsockopt_string(zmq.SUBSCRIBE, symbol)

In [7]:
times = list()
prices = list()

for _ in range(50):
    msg = socket.recv_string()
    t = datetime.now()
    times.append(t)
    _, price = msg.split()
    prices.append(float(price))
    fig.data[0].x = times
    fig.data[0].y = prices

In [8]:
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'e539c13b-2d7e-49de-a237-4b4ab0defa71',
 …

### Three Real-Time Streams

In [9]:
fig = go.FigureWidget()
fig.add_scatter(name=symbol)
fig.add_scatter(name='SMA1', line=dict(width=1, dash='dot'), mode='lines+markers')
fig.add_scatter(name='SMA2', line=dict(width=1, dash='dash'), mode='lines+markers')

FigureWidget({
    'data': [{'name': 'SYMBOL', 'type': 'scatter', 'uid': '2da4420c-f0b8-4d76-8f1d-9724b56241bc…

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame()
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df['SMA1'] = df[sym].rolling(5).mean()
    df['SMA2'] = df[sym].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['SMA1']
    fig.data[2].y = df['SMA2']

/tmp/ipykernel_90531/1587551658.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_90531/1587551658.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_90531/1587551658.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_90531/1587551658.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_90531/1587551658.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_90531/1587551658.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future

In [12]:
fig

FigureWidget({
    'data': [{'name': 'SYMBOL',
              'type': 'scatter',
              'uid': '2da4420c…

### Three Sub-Plots for Three Streams

In [13]:
from plotly.subplots import make_subplots

In [14]:
f = make_subplots(rows=3, cols=1, shared_xaxes=True)
f.append_trace(go.Scatter(name='SYMBOL'), row=1, col=1)
f.append_trace(go.Scatter(name='RETURN', line=dict(width=1, dash='dot'),
                  mode='lines+markers', marker={'symbol': 'triangle-up'}),
                  row=2, col=1)
f.append_trace(go.Scatter(name='MOMENTUM', line=dict(width=1, dash='dash'),
                  mode='lines+markers', marker={'symbol': 'x'}), row=3, col=1)

In [15]:
fig = go.FigureWidget(f)
fig

FigureWidget({
    'data': [{'name': 'SYMBOL',
              'type': 'scatter',
              'uid': '0d642603…

In [16]:
import numpy as np

In [17]:
df = pd.DataFrame()
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df['RET'] = np.log(df[sym] / df[sym].shift(1))
    df['MOM'] = df['RET'].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['RET']
    fig.data[2].y = df['MOM']

/tmp/ipykernel_90531/1063225289.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_90531/1063225289.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_90531/1063225289.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_90531/1063225289.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_90531/1063225289.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_90531/1063225289.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future

### Streaming Data as Bars

In [19]:
socket = context.socket(zmq.SUB)
socket.connect('tcp://0.0.0.0:5556')
socket.setsockopt_string(zmq.SUBSCRIBE, '')

In [20]:
for _ in range(5):
    msg = socket.recv_string()
    print(msg)

23.358 12.534 54.599 18.840 94.246 91.260 82.061 0.450
33.034 72.571 45.257 81.714 45.153 5.724 15.521 47.189
93.772 65.743 56.695 4.371 27.930 76.145 38.427 92.600
93.731 11.947 31.602 27.868 48.990 83.932 58.924 55.146
26.560 71.595 97.724 21.642 54.276 35.768 54.033 15.541


In [21]:
fig = go.FigureWidget()
fig.add_bar()
fig

FigureWidget({
    'data': [{'type': 'bar', 'uid': 'e8fa9503-22b0-48b2-87de-c9051a6314cb'}], 'layout': {'templ…

In [22]:
x = list('abcdefgh')
fig.data[0].x = x
for _ in range(25):
    msg = socket.recv_string()
    y = msg.split()
    y = [float(n) for n in y]
    fig.data[0].y = y